In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from typing import Dict, Text
from tensorflow import keras
import tensorflow_recommenders as tfrs
try:
    if not tf.config.list_physical_devices('GPU'):
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")
except:
    if not tf.test.is_gpu_available():
        assert tf.__version__ >= "2.0"
        print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
        if IS_COLAB:
            print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
        if IS_KAGGLE:
            print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import datetime as dt
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
try:
    if not tf.config.list_physical_devices('GPU'):
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)
except:
    if not tf.test.is_gpu_available():
        tf.random.set_seed(42)
    else:
        tf.random.set_random_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


In [2]:
trans_train = pd.read_csv('h-and-m-personalized-fashion-recommendations/transactions_train.csv')
trans_train['quantity']=1
master_df = trans_train[['customer_id','article_id','quantity']].astype(str)
master_df['quantity'] = master_df['quantity'].astype(float)
masterdf = master_df

In [3]:
### define interactions data and user data

### interactions 
### here we create a reference table of the user , item, and quantity purchased
interactions_dict = masterdf.groupby(['customer_id', 'article_id'])[ 'quantity'].sum().reset_index()

## we tansform the table inta a dictionary , which then we feed into tensor slices
# this step is crucial as this will be the type of data fed into the embedding layers
interactions_dict = {name: np.array(value) for name, value in interactions_dict.items()}
interactions = tf.data.Dataset.from_tensor_slices(interactions_dict)

## we do similar step for item, where this is the reference table for items to be recommended
items_dict = masterdf[['article_id']].drop_duplicates()
items_dict = {name: np.array(value) for name, value in items_dict.items()}
items = tf.data.Dataset.from_tensor_slices(items_dict)

## map the features in interactions and items to an identifier that we will use throught the embedding layers
## do it for all the items in interaction and item table
## you may often get itemtype error, so that is why here i am casting the quantity type as float to ensure consistency
interactions = interactions.map(lambda x: {
    'customer_id' : x['customer_id'], 
    'article_id' : x['article_id'], 
    'quantity' : float(x['quantity']),

})

items = items.map(lambda x: x['article_id'])

In [4]:
unique_item_titles = np.unique(np.concatenate(list(items.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(interactions.batch(1_000).map(lambda x: x["customer_id"]))))

In [5]:
### get unique item and user id's as a lookup table
unique_item_titles = np.unique(np.concatenate(list(items.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(interactions.batch(1_000).map(lambda x: x["customer_id"]))))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = interactions.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(60_000)
test = shuffled.skip(60_000).take(20_000)

In [6]:
class RetailModel(tfrs.Model):

    def __init__(self, user_model, item_model):
        super().__init__()
        
        ### Candidate model (item)
        ### This is Keras preprocessing layers to first convert user ids to integers, 
        ### and then convert those to user embeddings via an Embedding layer. 
        ### We use the list of unique user ids we computed earlier as a vocabulary:
        item_model = tf.keras.Sequential([
                                        tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=unique_item_titles, mask_token=None),
                                        tf.keras.layers.Embedding(len(unique_item_titles) + 1, embedding_dimension)
                                        ])
        ### we pass the embedding layer into item model
        self.item_model: tf.keras.Model = item_model
            
        ### Query model (users)    
        user_model = tf.keras.Sequential([
                                        tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=unique_user_ids, mask_token=None),
                                        # We add an additional embedding to account for unknown tokens.
                                        tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
                                        ])
        self.user_model: tf.keras.Model = user_model
        
        ### for retrieval model. we take top-k accuracy as metrics
        metrics = tfrs.metrics.FactorizedTopK(candidates=items.batch(128).map(item_model))
        
        # define the task, which is retrieval                                    )    
        task = tfrs.tasks.Retrieval(metrics=metrics)
       
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["customer_id"])
        # And pick out the movie features and pass them into the movie model,
        # getting embeddings back.
        positive_movie_embeddings = self.item_model(features["article_id"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_movie_embeddings)

In [7]:
### Fitting and evaluating

### we choose the dimensionality of the query and candicate representation.
embedding_dimension = 32

## we pass the model, which is the same model we created in the query and candidate tower, into the model
item_model = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.StringLookup(
                                vocabulary=unique_item_titles, mask_token=None),
                                tf.keras.layers.Embedding(len(unique_item_titles) + 1, embedding_dimension)
                                ])

user_model = tf.keras.Sequential([
                                tf.keras.layers.experimental.preprocessing.StringLookup(
                                vocabulary=unique_user_ids, mask_token=None),
                                # We add an additional embedding to account for unknown tokens.
                                tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
                                ])

model = RetailModel(user_model, item_model)

# a smaller learning rate may make the model move slower and prone to overfitting, so we stick to 0.1
# other optimizers, such as SGD and Adam, are listed here https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
model.compile(optimizer=tf.keras.optimizers.SGD())

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

## fit the model with ten epochs
model_hist = model.fit(cached_train, epochs=20)

#evaluate the model
model.evaluate(cached_test, return_dict=True)

Epoch 1/20
8/8 [==============================] - 118s 14s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.6667e-05 - factorized_top_k/top_10_categorical_accuracy: 1.6667e-05 - factorized_top_k/top_50_categorical_accuracy: 4.3333e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0011 - loss: 62067.2244 - regularization_loss: 0.0000e+00 - total_loss: 62067.2244
Epoch 2/20
8/8 [==============================] - 126s 16s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.6667e-05 - factorized_top_k/top_10_categorical_accuracy: 5.0000e-05 - factorized_top_k/top_50_categorical_accuracy: 7.1667e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0016 - loss: 62063.5365 - regularization_loss: 0.0000e+00 - total_loss: 62063.5365
Epoch 3/20
8/8 [==============================] - 117s 14s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_t

5/5 [==============================] - 41s 8s/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0012 - factorized_top_k/top_10_categorical_accuracy: 0.0016 - factorized_top_k/top_50_categorical_accuracy: 0.0047 - factorized_top_k/top_100_categorical_accuracy: 0.0066 - loss: 32588.3737 - regularization_loss: 0.0000e+00 - total_loss: 32588.3737


{'factorized_top_k/top_1_categorical_accuracy': 0.0002500000118743628,
 'factorized_top_k/top_5_categorical_accuracy': 0.0012499999720603228,
 'factorized_top_k/top_10_categorical_accuracy': 0.0016499999910593033,
 'factorized_top_k/top_50_categorical_accuracy': 0.004699999932199717,
 'factorized_top_k/top_100_categorical_accuracy': 0.006649999879300594,
 'loss': 29626.51171875,
 'regularization_loss': 0,
 'total_loss': 29626.51171875}

In [8]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 39s 8s/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0012 - factorized_top_k/top_10_categorical_accuracy: 0.0016 - factorized_top_k/top_50_categorical_accuracy: 0.0047 - factorized_top_k/top_100_categorical_accuracy: 0.0066 - loss: 32588.3737 - regularization_loss: 0.0000e+00 - total_loss: 32588.3737


{'factorized_top_k/top_1_categorical_accuracy': 0.0002500000118743628,
 'factorized_top_k/top_5_categorical_accuracy': 0.0012499999720603228,
 'factorized_top_k/top_10_categorical_accuracy': 0.0016499999910593033,
 'factorized_top_k/top_50_categorical_accuracy': 0.004699999932199717,
 'factorized_top_k/top_100_categorical_accuracy': 0.006649999879300594,
 'loss': 29626.51171875,
 'regularization_loss': 0,
 'total_loss': 29626.51171875}

In [9]:
unique_item_titles

array([b'108775015', b'108775044', b'108775051', ..., b'953450001',
       b'953763001', b'956217002'], dtype=object)

In [10]:
unique_user_ids 

array([b'00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
       b'0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa',
       b'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
       ...,
       b'ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264',
       b'ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a',
       b'ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241'],
      dtype=object)

In [11]:
train

<TakeDataset shapes: {customer_id: (), article_id: (), quantity: ()}, types: {customer_id: tf.string, article_id: tf.string, quantity: tf.float32}>

In [12]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(items.batch(100).map(lambda items: (items,model.item_model(items))))

In [13]:
j = '00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657'
_, titles = index(tf.constant([j]),k=12)
print(f"Recommendations for user %s: {titles[0]}" %(j))

Recommendations for user 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657: [b'749699002' b'599580049' b'719031001' b'739144007' b'753214007'
 b'537688015' b'679853009' b'677930066' b'636323002' b'706016001'
 b'805376003' b'399223001']
